# Singularityのロード

---

構築したOpenHPC環境で[Singularity](https://sylabs.io/singularity/)がデフォルトでロードされるように設定します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境の各ノードに対してAnsibleで操作できるように設定されている

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

各ノードに対してAnsibleによる操作が行えることを確認します。操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name =

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping

## 設定変更

Singularity(version:3.7.1)をデフォルトで利用できるようにするために `/etc/profile.d/` の設定ファイルを変更します。

OpenHPC環境では、デフォルトでは[Singularity](https://sylabs.io/singularity/)が利用できるようになっていません。ここでは`/etc/profile.d`/にある[Lmod](https://lmod.readthedocs.io/en/latest/index.html)の設定ファイルを変更操作して Singularity を利用できるようにします。

> OpenHPCでは複数の開発ツールやライブラリから実際に利用するものを選択するためにLmodを使用しています。

まずLmodのコマンド`module`を実行して、OpenHPC環境で利用可能なモジュールの一覧を表示してみます。

In [ ]:
!ansible {ugroup_name} -m shell -a 'module avail'

一覧表示されたモジュールのなかで `(L)` のマークがついているものはLmodがロード済のもので、実行可能な状態になっています。

ロードされたモジュールの一覧を表示してみます。

In [ ]:
!ansible {ugroup_name} -m shell -a 'module list'

Lmod がデフォルトでロードするモジュールは `/etc/profile.d/` にある `lmod.sh`, `lmod.csh` で設定されています。Singularityをロードするように設定ファイルを書き換えます。

In [ ]:
# 書き換え対象のファイル
files = ['/etc/profile.d/lmod.sh', '/etc/profile.d/lmod.csh']

# 追加する行
line = 'module try-add singularity'

for file in files:
    !ansible {ugroup_name} -m lineinfile -b -a \
        'path={file} backup=yes line="{line}" regexp="module\s+try-add\s+singularity"'

設定が変更されたことを確認するために、ロードモジュールの内容を再度確認してみます。

In [ ]:
!ansible {ugroup_name} -m shell -a 'module list'

以下のように `singularity/3.7.1` がロードモジュールに追加されていれば、設定変更が行われたことが確認できます。
```
Currently Loaded Modules:
  1) autotools
  2) prun/2.2
  3) gnu9/9.4.0
  4) hwloc/2.5.0
  5) ucx/1.11.2
  5) libfabric/1.13.0
  6) openmpi4/4.1.1
  7) ohpc
  8) singularity/3.7.1
```

Singularityがロードされていることを確認します。

> ロードされていない場合は、次のセルを実行するとエラーになります。

In [ ]:
!ansible {ugroup_name} -m shell -a 'module is-loaded singularity'

`singularity`コマンドが実行できることを確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a 'singularity version'